# Presenting monthly updates results

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook

import warnings
warnings.filterwarnings('ignore')

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *

# Packages from this repository, Tools folder
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
from FetchData import FetchData, RetrieveFromList
from ViewsEstimators import *

In [ ]:
# Common parameters:

dev_id = 'Fatalities002'
run_id = dev_id 
EndOfHistory = 509
prod_id = '2022_04_t01'
RunGeneticAlgo = False
level = 'cm'
WriteToOverleaf = False

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

# Specifying partitions

calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

# Specifying paths - note these have to be set to conform to individual setups!
#note the home path has been automated with the os.path.expanduser function

import os
home = os.path.expanduser("~")
Mydropbox = home + '/Dropbox (ViEWS)/ViEWS/'
#localgitpath = '/Users/havardhegre/views3/'


In [ ]:
# Read in future predictions as given by the parameters above

predstore_future = level +  '_' + 'genetic_ensemble' + '_f' + str(EndOfHistory)
predstore_future_dich = level +  '_' + 'genetic_ensemble' + '_dich_f' + str(EndOfHistory)

predictions_df = pd.DataFrame.forecasts.read_store(predstore_future, run=dev_id)
predictions_dich_df = pd.DataFrame.forecasts.read_store(predstore_future_dich, run=dev_id)


# Maps country-month level

In [ ]:
#import the necessary programs for mapping
import geopandas as gpd
import sqlalchemy as sa
from ingester3.config import source_db_path

from views_mapper2.mapper2 import *

from views_mapper2.dictionary_writer import *
from views_mapper2.BBoxWriter import *
from views_mapper2.label_writer import *

Note: ask Angelica and Havard where in the MonthlyUpdates folder files should be written to

In [ ]:
#specify the directory for saving output
#change the directory name to correspond to the run description

foldername = 'mr_testing_output_options'
folderpath = Mydropbox + 'DataReleases/MonthlyUpdates/' + foldername

if not os.path.exists(folderpath):
    os.mkdir(folderpath)
    print('Directory', folderpath, "created")
else:
    print('Directory', folderpath, 'already exists')

In [ ]:
#obtaining the geometry for the data
engine = sa.create_engine(source_db_path)
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

Note: ask Angelica and Havard about attribution of the Stamen tiles, attribution placed in the mapper2 read me portion. Should it also be placed in the main monthlyupdate read me if the tiles are used?

In [ ]:
#create or choose an appropriate dictionary for the data
#standard dictionaries are defined by the views_mapper2.dictionary_writer
#custom ones can be created with dictionary_writer functions

custom_dictionary_1p = log1p_dict([0, 1, 3, 10, 30, 100, 300])
display(dictionary_stand_1p, custom_dictionary_1p)

Different options are presented for review by H and A

## AME (Africa Middle East) maps

In [ ]:
#standard map at country_month level
#recursive run for months of interest

#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
bbox_run = bbox_from_cid('ame')
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'_standard'
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=bbox_run,
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        m.save(savefile)

In [ ]:
#standard map at country_month level, but now with double logged scale
#recursive run for months of interest

#prep data
data= predictions_df.copy()
data['step_combined_2p'] = np.log1p(data['step_combined'])
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined_2p']
var_labels = ['step combined double logged']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
bbox_run = bbox_from_cid('ame')
map_dictionary_run = dictionary_stand_2p
scale_for_run_name_string = '_log2_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'_standard'
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=bbox_run,
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        m.save(savefile)

In [ ]:
#standard map at country_month level, tiles for names

#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
bbox_run = bbox_from_cid('ame')
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'_tiles'
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=bbox_run,
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            background = 'grey',
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        m.save(savefile)

In [ ]:
#standard map at country_month level, naming strategy (tile free)
#this method has issues with overlapping names of countries
#solution for label adjustment has not yet been achieved 
#cleaner appearance than tiles


#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()
gdf_c = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
bbox_run = bbox_from_cid('ame')
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'_no_tiles_v1'
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=bbox_run,
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            cmap='rainbow',
            edgecolor="grey",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        country_id_list = gdf.loc[i].drop_duplicates(['name']).index.values.tolist()
        gdf_limited = gdf_c.loc[(gdf_c['country_id'].isin(country_id_list))]
        gdf_limited2 = gdf_limited.loc[((gdf_c['in_africa'] == 1)|(gdf_c['in_me']==1))]
    
        gdf_limited2['coords'] = gdf_limited2['geom'].apply(lambda x: x.representative_point().coords[:])
        gdf_limited2['coords'] = [coords[0] for coords in gdf_limited2['coords']]

        ax = m.ax
        gdf_limited2.plot(ax=ax, linewidth = 0.5, edgecolor = 'grey', facecolor='None')
        gdf_limited2.apply(lambda x: ax.annotate(text=x['name'], xy=x.geom.centroid.coords[0], ha='center', size = 5), axis=1)

        m.save(savefile)

In [ ]:
#standard map at country_month level, naming strategy (tile free)
#attempt to resolve overlap by reducing country names given

#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()
gdf_c = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
bbox_run = bbox_from_cid('ame')
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'_no_tiles_v2'
        m=Mapper2(
            width=10,
            height=10,
            frame_on=True,
            title=title_def,
            bbox=bbox_run,
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            cmap='rainbow',
            edgecolor="grey",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        country_id_list = gdf.loc[i].drop_duplicates(['name']).index.values.tolist()
        countries_spaced = lambda x: x % 2 == 0
        country_id_list2 = list(filter(countries_spaced, country_id_list))
        
        gdf_limited = gdf_c.loc[(gdf_c['country_id'].isin(country_id_list2))]
        gdf_limited2 = gdf_limited.loc[((gdf_c['in_africa'] == 1)|(gdf_c['in_me']==1))]
    
        gdf_limited2['coords'] = gdf_limited2['geom'].apply(lambda x: x.representative_point().coords[:])
        gdf_limited2['coords'] = [coords[0] for coords in gdf_limited2['coords']]

        ax = m.ax
        gdf_limited2.plot(ax=ax, linewidth = 0.5, edgecolor = 'grey', facecolor='None')
        gdf_limited2.plot(ax=ax, linewidth = 0.1, facecolor='None')
        gdf_limited2.apply(lambda x: ax.annotate(text=x['name'], xy=x.geom.centroid.coords[0], ha='center', size = 5), axis=1)

        m.save(savefile)

## global maps

In [ ]:
#global maps

#standard map at country_month level
#recursive run for months of interest

#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'global_standard'
        m=Mapper2(
            width=40,
            height=10,
            frame_on=True,
            title=title_def
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        m.save(savefile)

In [ ]:
#standard map at country_month level, but now with double logged scale
#recursive run for months of interest

#prep data
data= predictions_df.copy()
data['step_combined_2p'] = np.log1p(data['step_combined'])
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined_2p']
var_labels = ['step combined double logged']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
map_dictionary_run = dictionary_stand_2p
scale_for_run_name_string = '_log2_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'global_standard'
        m=Mapper2(
            width=40,
            height=10,
            frame_on=True,
            title=title_def
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        m.save(savefile)

In [ ]:
#standard map at country_month level, tiles for names
#tiles not always stable, occasional failure to associate

#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'global_tiles'
        m=Mapper2(
            width=40,
            height=10,
            frame_on=True,
            title=title_def
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            background = 'grey',
            cmap='rainbow',
            edgecolor="black",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        m.save(savefile)

In [ ]:
#standard map at country_month level, naming strategy (tile free)
#attempt to resolve overlap by reducing country names given

#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()
gdf_c = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        variable_def = key
        title_def = 'Predicted fatalities in ' + str(vid2date(i)) + ', ' + value
        textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
        savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'global_no_tiles_v2'
        m=Mapper2(
            width=40,
            height=10,
            frame_on=True,
            title=title_def
        ).add_layer(
            gdf=gdf.loc[i],
            map_dictionary=map_dictionary_run,
            cmap='rainbow',
            edgecolor="grey",
            linewidth=0.5,
            column=variable_def
        ).add_views_textbox(
            text=textbox,
            textsize=16)
        
        country_id_list = gdf.loc[i].drop_duplicates(['name']).index.values.tolist()
        countries_spaced = lambda x: x % 3 == 0
        country_id_list2 = list(filter(countries_spaced, country_id_list))
        
        gdf_limited2 = gdf_c.loc[(gdf_c['country_id'].isin(country_id_list2))]
    
        gdf_limited2['coords'] = gdf_limited2['geom'].apply(lambda x: x.representative_point().coords[:])
        gdf_limited2['coords'] = [coords[0] for coords in gdf_limited2['coords']]

        ax = m.ax
        gdf_limited2.plot(ax=ax, linewidth = 0.5, edgecolor = 'grey', facecolor='None')
        gdf_limited2.plot(ax=ax, linewidth = 0.1, facecolor='None')
        gdf_limited2.apply(lambda x: ax.annotate(text=x['name'], xy=x.geom.centroid.coords[0], ha='center', size = 5), axis=1)

        m.save(savefile)

## Country specific 

In [ ]:
#standard map at country_month level
#recursive run for months of interest and countries of interest
#ensure spelling is correct


#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
countries_wanted = {'Togo', 'Ukraine'}
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        for j in countries_wanted:
            variable_def = key
            title_def = 'Predicted fatalities in ' + str(j) + ' '+ str(vid2date(i)) + ', ' + value
            textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
            savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'_standard' + '_' + str(j)
            bbox_run = bbox_from_cid(name_mid2cid(j, i))
            m=Mapper2(
                width=10,
                height=10,
                frame_on=True,
                title=title_def,
                bbox=bbox_run
            ).add_layer(
                gdf=gdf.loc[i],
                map_dictionary=map_dictionary_run,
                cmap='rainbow',
                edgecolor="black",
                linewidth=0.5,
                column=variable_def
            ).add_views_textbox(
                text=textbox,
                textsize=10)
        
            m.save(savefile)

In [ ]:
#country_month_level using tiles
#recursive run for months of interest and countries of interest
#ensure spelling is correct


#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
countries_wanted = {'Togo', 'Ukraine'}
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        for j in countries_wanted:
            variable_def = key
            title_def = 'Predicted fatalities in ' + str(j) + ' '+ str(vid2date(i)) + ', ' + value
            textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
            savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'_tiles' + '_' + str(j)
            bbox_run = bbox_from_cid(name_mid2cid(j, i))
            m=Mapper2(
                width=10,
                height=10,
                frame_on=True,
                title=title_def,
                bbox=bbox_run
            ).add_layer(
                gdf=gdf.loc[i],
                map_dictionary=map_dictionary_run,
                background = 'grey',
                cmap='rainbow',
                edgecolor="black",
                linewidth=0.5,
                column=variable_def
            )
        
            m.save(savefile)

In [ ]:
#standard map at country_month level
#recursive run for months of interest and countries of interest
#ensure spelling is correct


#prep data
data= predictions_df.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#specify months of interest and variables of interest, change this part
list_of_months = [510]
list_of_variables = ['step_combined']
var_labels = ['step combined']
fact_dict =dict(zip(list_of_variables, var_labels))

#specify other run parameters
countries_wanted = {'Togo', 'Ukraine'}
map_dictionary_run = dictionary_stand_1p
scale_for_run_name_string = '_log_scale'

for i in list_of_months:
    for key, value in fact_dict.items():
        for j in countries_wanted:
            variable_def = key
            title_def = 'Predicted fatalities in ' + str(j) + ' '+ str(vid2date(i)) + ', ' + value
            textbox = 'Name: '+ key + '_' + str(i) + '\nMonth:' + str(i)
            savefile = folderpath + '/'+ key + '_' + str(i)+scale_for_run_name_string+'_no_tiles' + '_' + str(j)
            bbox_run = bbox_from_cid(name_mid2cid(j, i))
            m=Mapper2(
                width=10,
                height=10,
                frame_on=True,
                title=title_def,
                bbox=bbox_run
            ).add_layer(
                gdf=gdf.loc[i],
                map_dictionary=map_dictionary_run,
                cmap='rainbow',
                edgecolor="grey",
                linewidth=0.5,
                column=variable_def
            ).add_views_textbox(
                text=textbox,
                textsize=10)
            
            country_id_list = gdf.loc[i].drop_duplicates(['name']).index.values.tolist()
            countries_spaced = lambda x: x % 1 == 0
            country_id_list2 = list(filter(countries_spaced, country_id_list))
        
            gdf_limited2 = gdf_c.loc[(gdf_c['country_id'].isin(country_id_list2))]
    
            gdf_limited2['coords'] = gdf_limited2['geom'].apply(lambda x: x.representative_point().coords[:])
            gdf_limited2['coords'] = [coords[0] for coords in gdf_limited2['coords']]

            ax = m.ax
            gdf_limited2.plot(ax=ax, linewidth = 0.5, edgecolor = 'grey', facecolor='None')
            gdf_limited2.plot(ax=ax, linewidth = 0.1, facecolor='None')
            gdf_limited2.apply(lambda x: ax.annotate(text=x['name'], xy=x.geom.centroid.coords[0], ha='center', size = 8), axis=1)
        
            m.save(savefile)